In [1]:
import numpy as np
import pandas as pd
import joblib
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df_attacks = pd.read_csv("C:/Users/kisho/Documents/GitHub/Risk_Quantification/Logged_Output/attack_desc.csv")
print(df_attacks.info)
cv = joblib.load("C:/Users/kisho/Documents/GitHub/Risk_Quantification/Risk_Quantifier/Criteria_Impact/cv.pkl")
    

<bound method DataFrame.info of                                              Payload  \
0               <script>alert(1)</script> dfd login    
1              && perl -e 'print ""X""x4096'  login    
2  admin' and substring(password/text(),1,1)='7  ...   

                  Attack Type  \
0  CROSS-SITE SCRIPTING (XSS)   
1           COMMAND INJECTION   
2               SQL INJECTION   

                                  Attack Description Confidentiality Impact  \
0  This is a payload for an XSS (Cross-Site Scrip...                    LOW   
1  The given payload "&& perl -e 'print ""X""x409...                    LOW   
2  This attack is an example of Blind SQL Injecti...                   HIGH   

  Integrity Impact Availability Impact  
0              LOW                NONE  
1              LOW                NONE  
2             NONE                NONE  >


In [3]:
conf_model = joblib.load("C:/Users/kisho/Documents/GitHub/Risk_Quantification/Risk_Quantifier/Criteria_Impact/ens_conf.pkl")
int_model = joblib.load("C:/Users/kisho/Documents/GitHub/Risk_Quantification/Risk_Quantifier/Criteria_Impact/ens_int.pkl")
avail_model = joblib.load("C:/Users/kisho/Documents/GitHub/Risk_Quantification/Risk_Quantifier/Criteria_Impact/ens_avail.pkl")

In [4]:
conf_arr = []
int_arr = []
avail_arr = []
payload_list = []
desc_list = []
for index, row in df_attacks.iterrows():
    current_desc = str(row['Attack Description'])
    payload_list.append(row['Payload'])
    desc_list.append(current_desc)
    new_X = cv.transform([current_desc])
    current_conf = conf_model.predict(new_X)[0]
    current_int = int_model.predict(new_X)[0]
    current_avail = avail_model.predict(new_X)[0]
    conf_arr.append(current_conf)
    int_arr.append(current_int)
    avail_arr.append(current_avail)
    


In [5]:
conf_arr = np.array(conf_arr)
int_arr = np.array(int_arr)
avail_arr = np.array(avail_arr)
df_attacks['Confidentiality Impact'] = conf_arr
df_attacks['Integrity Impact'] = int_arr
df_attacks['Availability Impact'] = avail_arr
    

In [6]:
df_attacks = df_attacks.reset_index(drop=True)
df_attacks.info

<bound method DataFrame.info of                                              Payload  \
0               <script>alert(1)</script> dfd login    
1              && perl -e 'print ""X""x4096'  login    
2  admin' and substring(password/text(),1,1)='7  ...   

                  Attack Type  \
0  CROSS-SITE SCRIPTING (XSS)   
1           COMMAND INJECTION   
2               SQL INJECTION   

                                  Attack Description Confidentiality Impact  \
0  This is a payload for an XSS (Cross-Site Scrip...                    LOW   
1  The given payload "&& perl -e 'print ""X""x409...                    LOW   
2  This attack is an example of Blind SQL Injecti...                   HIGH   

  Integrity Impact Availability Impact  
0              LOW                NONE  
1              LOW                NONE  
2             NONE                NONE  >

In [8]:
priority_mapping = {
    "HIGH": 10,
    "LOW": 5,
    "NONE": 1
}
conf_impact = np.array(df_attacks['Confidentiality Impact'].values)
int_impact = np.array(df_attacks['Integrity Impact'].values)
avail_impact = np.array(df_attacks['Availability Impact'].values)
criteria_weights = np.array([0.6, 0.2, 0.2])


In [9]:
risk_level_arr = []
risk_term_arr = []
i = 0
for index, row in df_attacks.iterrows():
    curr_conf = priority_mapping.get(conf_impact[i])
    curr_int = priority_mapping.get(int_impact[i])
    curr_avail = priority_mapping.get(avail_impact[i])

    impact_matrix = np.array([[1, curr_conf, curr_avail],
                          [1/curr_conf, 1, curr_int],
                          [1/curr_avail, 1/curr_int, 1]])
    
    print(impact_matrix)
    

    risk_level = np.sum(impact_matrix * criteria_weights, axis=1)
    risk_level = np.sum(risk_level) / np.sum(criteria_weights)

    if risk_level < 3.33:
        risk_term = 'Low'
    elif risk_level < 6.67:
        risk_term = 'Medium'
    else:
        risk_term = 'High'

    risk_term_arr.append(risk_term)
    print('Overall risk level:', risk_level)
    print('Linguistic term:', risk_term)

    i = i+1

    
    
    

[[1.  5.  1. ]
 [0.2 1.  5. ]
 [1.  0.2 1. ]]
Overall risk level: 3.96
Linguistic term: Medium
[[1.  5.  1. ]
 [0.2 1.  5. ]
 [1.  0.2 1. ]]
Overall risk level: 3.96
Linguistic term: Medium
[[ 1.  10.   1. ]
 [ 0.1  1.   1. ]
 [ 1.   1.   1. ]]
Overall risk level: 4.26
Linguistic term: Medium


In [10]:
risk_term_arr = np.array(risk_term_arr)
df_attacks['Overall Risk Level'] = risk_term_arr
df_attacks.info

<bound method DataFrame.info of                                              Payload  \
0               <script>alert(1)</script> dfd login    
1              && perl -e 'print ""X""x4096'  login    
2  admin' and substring(password/text(),1,1)='7  ...   

                  Attack Type  \
0  CROSS-SITE SCRIPTING (XSS)   
1           COMMAND INJECTION   
2               SQL INJECTION   

                                  Attack Description Confidentiality Impact  \
0  This is a payload for an XSS (Cross-Site Scrip...                    LOW   
1  The given payload "&& perl -e 'print ""X""x409...                    LOW   
2  This attack is an example of Blind SQL Injecti...                   HIGH   

  Integrity Impact Availability Impact Overall Risk Level  
0              LOW                NONE             Medium  
1              LOW                NONE             Medium  
2             NONE                NONE             Medium  >

In [11]:
df_attacks.to_csv("C:/Users/kisho/Documents/GitHub/Risk_Quantification/Logged_Output/attack_desc.csv",index=False)